In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from scipy.stats import uniform

import weakref 

from bayes_opt import BayesianOptimization
#from root_pandas import read_root


from data_cleaning import clean_df
from KFPF_lambda_cuts import KFPF_lambda_cuts
from plot_tools import AMS, preds_prob, plot_confusion_matrix, plt_sig_back
import tree_importer 
import uproot


#To save some memory we will delete unused variables
class TestClass(object): 
    def check(self): 
        print ("object is alive!") 
    def __del__(self): 
        print ("object deleted") 
        
from concurrent.futures import ThreadPoolExecutor
executor = ThreadPoolExecutor(7)

import gc
import ROOT
from ROOT import TFile, TTree

In [ ]:

            
ROOT.gInterpreter.Declare('''
void Print_start(TCanvas* c) {
c->Print("hists.pdf(","pdf");
}
''')

ROOT.gInterpreter.Declare('''
void Print_continue(TCanvas* c) {
c->Print("hists.pdf","pdf");
}
''')

ROOT.gInterpreter.Declare('''
void Print_end(TCanvas* c) {
c->Print("hists.pdf)","pdf");
}
''')


ROOT.gInterpreter.Declare('''
TH2F* Divide2DHisto(const TH2F* h1, const TH2F* h2, const TString& name) {

  if(h1->GetNbinsX() != h2->GetNbinsX() || h1->GetNbinsY() != h2->GetNbinsY()) {
    throw std::runtime_error("Histograms should have the same number of bins!");
  }

  TH2F* res = dynamic_cast<TH2F*>(h1->Clone(name));
  res->Divide(h2);

  for(int i = 1; i <= h1->GetNbinsX(); ++i){
    for(int j = 1; j <= h1->GetNbinsY(); ++j) {
      const auto a = h1->GetBinContent(i, j);
      const auto b = h2->GetBinContent(i, j);
      const auto da = h1->GetBinError(i, j);
      const auto db = h2->GetBinError(i, j);
      if(a == 0 || b == 0) {
        continue;
      }
      double error = res->GetBinContent(i, j) * sqrt( ((da * da) / (a * a)) + ((db * db) / (b * b)) );
      res->SetBinError(i, j, error);
    }
  }
  return res;
}
''')

ROOT.gInterpreter.Declare('''
  auto legend_draw(TH1D* mc, TH1D* reco){
  auto legend = new TLegend(0.3,0.4,0.4,0.5);
  legend->AddEntry(mc,"Simulated","ep");
  legend->AddEntry(reco,"Efficiency corrected reconstructed","ep");
  legend -> SetLineWidth (0);
  legend->SetTextSize(0.03);
  legend->Draw();
  }
  ''')

ROOT.gInterpreter.Declare('''
    auto ratio_plot(TH1D* h1, TH1D* h2, char* a, char* option){
    auto rp = new TRatioPlot(h1, h2, option);
    rp->Draw(a);
    return rp;
    }
      ''')

In [ ]:
a = ['index', 'chi2geo','chi2primneg', 'chi2primpos','distance', 'ldl','l','cosine_first', 'cosine_second', 'chi2_topo', 'cosine_topo',
 'mass', 'pT', 'b', 'eta', 'p', 'phi', 'rapidity', 'vtx_chi2_first', 'vtx_chi2_second', 'z_first', 'z_second', 'z_smaller', 'M', 'issignal', 'nhits_mvd_first',
 'nhits_mvd_second', 'nhits_mvd_sum', 'nhits_tot_first', 'nhits_tot_second', 'nhits_tot_sum']

#1st sample
df_dcm1 =  uproot.open('/home/shahid/Mount/gsi/u/Mount/lustre/khan/cbmsoft/at_tree_plainer/install/bin/dcm/c_0_pt_0_9_y_0_9_M_200_250.root:plain_tree',decompression_executor=executor,
                                  interpretation_executor=executor).arrays(library='pd',decompression_executor=executor,
                                  interpretation_executor=executor)
df_dcm1.columns = a
df_dcm1= df_dcm1[(df_dcm1['issignal']>0)]

#2nd sample
df_dcm2= uproot.open('/home/shahid/Mount/gsi/u/Mount/lustre/khan/cbmsoft/at_tree_plainer/install/bin/urqmd/c_0_pt_0_9_y_0_9_M_200_250.root:plain_tree',decompression_executor=executor,
                                  interpretation_executor=executor).arrays(library='pd',decompression_executor=executor,
                                  interpretation_executor=executor)
df_dcm2.columns = a
df_dcm2= df_dcm2[(df_dcm2['issignal']>0) ]

#fill histograms
import ROOT
from ROOT import TFile, TTree
bins =15
h2d_dcm1 = ROOT.TH2F("Mc DCM", "Reconstructed_{DCM1}/Simulated_{DCM1}", bins,0,3,bins,0,3)
for i in range(0,len( df_dcm1['rapidity'])):
    h2d_dcm1.Fill( df_dcm1['rapidity'].iloc[i],df_dcm1['pT'].iloc[i])

h2d_dcm2 = ROOT.TH2F("urqmd", "Reconstructed_{DCM2}/Simulated_{DCM2} ", bins,0,3,bins,0,3)
for i in range(0,len(df_dcm2['rapidity'])):
    h2d_dcm2.Fill(df_dcm2['rapidity'].iloc[i],df_dcm2['pT'].iloc[i])

In [ ]:
file = TFile("/home/shahid/cbmsoft/Cut_optimization/uncut_data/Project/dcm/m_200_250_dcm_minus_10_100_z_all.root")
MC_DCM1=file.Get("SimParticles_McLambda/SimParticles_rapidity_SimParticles_pT_McLambda;1")
file1 = ROOT.TFile("/home/shahid/cbmsoft/Cut_optimization/uncut_data/Project/urqmd/m_200_250_urqmd_minus_10_100_z_all.root")
MC_URQMD=file1.Get("SimParticles_McLambda/SimParticles_rapidity_SimParticles_pT_McLambda;1")
#h2d_dcm1.Divide(MC_DCM1)
h2d_dcm1_err_updated = ROOT.Divide2DHisto(h2d_dcm1, MC_DCM1,"dcm_eff")
#h2d_dcm2.Divide(h2d_dcm1_err_updated)
#h2d_urqmd.Divide(MC_URQMD)
h2d_urqmd_eff_correc = ROOT.Divide2DHisto(h2d_dcm2, h2d_dcm1_err_updated,"dcm_eff_corr")

In [ ]:
ROOT.gInterpreter.Declare('''
auto ratio_p(TH1D *h1, TH1D *h2, TCanvas *c1) {
   h1->GetXaxis()->SetLabelFont(63); //font in pixels
   h1->GetXaxis()->SetLabelSize(16); //in pixels
   h1->GetYaxis()->SetLabelFont(63); //font in pixels
   h1->GetYaxis()->SetLabelSize(16); //in pixels
   TLine* line = new TLine (0,1 ,3 ,1);
   line -> SetLineColor ( kBlack );
   TPad *pad1 = new TPad("pad1","pad1",0,0.3,1,1);
   pad1->SetBottomMargin(0);pad1->SetTopMargin(0.1);pad1->SetLeftMargin(0.13);
   pad1->Draw();
   pad1->cd();
   h1->DrawCopy("pe1");
   h2->Draw("pe1,same");
   c1->cd();
   TPad *pad2 = new TPad("pad2","pad2",0,0,1,0.3);
   pad2->SetBottomMargin(0.3);pad2->SetTopMargin(0);pad2->SetLeftMargin(0.13);
   pad2->Draw();
   pad2->cd();
   h1->Sumw2();
   h1->SetStats(0);
   h1->Divide(h2);
   h1->SetMarkerStyle(21);
   h1 -> GetYaxis () -> SetTitle ("Ratio");
   h1->SetTitle("");
   h1 -> GetXaxis () -> SetTitle("y_{Lab}");
   h1 -> GetXaxis () -> SetTitleSize (0.15);
   h1 -> GetXaxis () -> SetTitleOffset (.7);
   h1 -> GetYaxis () -> SetTitleSize (0.15);
   h1 -> GetYaxis () -> SetTitleOffset (.3);
   h1->Draw("ep");
   line->Draw("SAME");
   c1->cd();
   return c1;
}
''')

In [ ]:
c1 = ROOT.TCanvas("","",500,500)
for i in range(0,15):
    bin1 = bin2 =i
    ROOT.gStyle.SetPaintTextFormat("4.2f")
    corr_urqmd_y = h2d_urqmd_eff_correc.ProjectionX("corr_urqmd_y",bin1,bin2)
    corr_urqmd_y . SetTitle("Reconstructed_{URQMD} x (Reconstructed_{DCM}/Simulated_{DCM})")
    corr_urqmd_y.SetTitleOffset(0.2);
    corr_urqmd_y.GetYaxis().SetTitleOffset(.9)
    corr_urqmd_y.GetYaxis().SetTitleSize (0.07)
    corr_urqmd_y.GetYaxis().SetTitle("Counts")
    corr_urqmd_y . SetStats (0)
    corr_urqmd_y . SetLineColor(ROOT.kRed)
    corr_urqmd_y . SetMarkerColor(ROOT.kRed)
    corr_urqmd_y . SetLineWidth(2)
    corr_urqmd_y . SetMarkerStyle(21)

    mc_urqmd_y = MC_URQMD.ProjectionX("mc_urqmd_y", bin1,bin2)

    mc_urqmd_y . SetLineWidth(2)
    mc_urqmd_y . SetMarkerStyle(22)
    mc_urqmd_y . SetLineColor(ROOT.kBlue)
    mc_urqmd_y . SetMarkerColor(ROOT.kBlue)
    
    c = ROOT.ratio_p(corr_urqmd_y,mc_urqmd_y,c1)
    c.Draw()
    #c.Clear()
    #c.Draw()
    #rp = ROOT.ratio_plot(mc_urqmd_y,corr_urqmd_y,"pe1","difference")
    #rp.GetYaxis().SetTitle("ratio")
    #mc_urqmd_y . Draw("pe1")
#    corr_urqmd_y . Draw("pe1same")
#    mc_urqmd_y . Draw("pe1same")
    ROOT.legend_draw( mc_urqmd_y, corr_urqmd_y)
    #mc_urqmd_y.GetYaxis().SetTitle("Counts")
#    
#    ROOT.Print_continue(c)
    if (i==1):
        ROOT.Print_start(c)
    if (i==14):
        ROOT.Print_end(c)
    else:
        ROOT.Print_continue(c)


    #h2d_urqmd.Draw("TEXT SAME")



In [ ]:
def draw_2d(hist,title,max_range):
    ROOT.gStyle.SetPaintTextFormat("4.2f")
    c = ROOT.TCanvas("","")
    c.Draw()
    hist.SetAxisRange(0, max_range, "Z");
    hist.SetTitle(title)
    hist.GetYaxis().SetTitle("p_{T}")
    hist.GetXaxis().SetTitle("y_{LAB}")
    hist.SetStats (0)
    hist.Draw("colz")
    hist.Draw("TEXT SAME")
    return c


In [ ]:
h2d_urqmd_eff_correc_eff = ROOT.Divide2DHisto(h2d_urqmd_eff_correc, MC_URQMD,"dcm_eff_corr")
c =draw_2d(h2d_urqmd_eff_correc_eff,"#frac{Reconstructed_{URQMD}}{Simulated_{URQMD}} x (Reconstructed_{DCM}/Simulated_{DCM})",1.2)
c.Print("hists.png")

In [ ]:
h2d_eff_urqmd = ROOT.Divide2DHisto(h2d_dcm2, MC_URQMD,"")
c = draw_2d(h2d_eff_urqmd,"#frac{Reconstructed_{URQMD}}{Simulated_{URQMD}}",0.6)
c.Print("hists_urqmd.png")

In [ ]:
h2d_eff_dcm = ROOT.Divide2DHisto(h2d_dcm1, MC_DCM1,"")
c = draw_2d(h2d_eff_dcm,"#frac{Reconstructed_{DCM}}{Simulated_{DCM}}",0.6)
c.Print("hists_dcm.png")

In [ ]:
h2d_eff_by_eff = ROOT.Divide2DHisto(h2d_eff_urqmd, h2d_eff_dcm,"#epsilon_{URQMD}/#epsilon_{DCM}")
c = draw_2d(h2d_eff_by_eff,"#epsilon_{URQMD}/#epsilon_{DCM}",1.2)
c.Print("hists_eff_by_ef.png")

In [ ]:
h_mass = ROOT.TH1D("h_mass", "Reconstructed_{DCM1}", 50,1.1,1.13)
for i in range(0,len( df_dcm1['mass'])):
    h_mass.Fill( df_dcm1['mass'].iloc[i])

In [ ]:
c.Draw()
h_mass.Draw()

In [ ]:
Double_t DSCB(Double_t *x, Double_t *par)
{
  Double_t mu_shift = 1.1157;
  Double_t factor = par[0];
  Double_t mu = par[1];
  Double_t sigma = par[2];
  Double_t a1 = par[3];
  Double_t n1 = par[4];
  Double_t a2 = par[5];
  Double_t n2 = par[6];
  
  Double_t u = (x[0]-mu_shift-mu)/sigma;
  Double_t A1 = TMath::Power(n1/a1, n1)*TMath::Exp(-a1*a1/2);
  Double_t A2 = TMath::Power(n2/a2, n2)*TMath::Exp(-a2*a2/2);
  Double_t B1 = n1/a1 - a1;
  Double_t B2 = n2/a2 - a2;
  
  if(u<-a1)
    return factor*A1*TMath::Power((B1-u), -n1);
  else if(u>=-a1 && u<a2)
    return factor*TMath::Exp(-u*u/2);
  else if(u>=a2)
    return factor*A2*TMath::Power((B2+u), -n2);
  else
    return -1.;
}